20230619 = Apenas treinamento, crossover + semantica, 100k avaliações, 500 população, <uop> com tudo

standard

In [7]:
import os
import pandas as pd
import plot as plot
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

directory = "results/"
dfs = []

for root, subdirectories, files in os.walk(directory):
    for file in files:
        if file.endswith("res.csv"):
            df = pd.read_csv(os.path.join(root, file), sep = ";")
            # print(root, file, os.path.join(root, file))
            # print(df)
            df['problem'] = [root.split('/')[1] for i in range(len(df))]
            df['amount'] = [root.split('/')[2] for i in range(len(df))]
            df['variation'] = [root.split('/')[3] for i in range(len(df))]
            df['seed_file'] = [str(seed) + '.hist' for seed in df.seed]
            dfs.append(df)
            
results = pd.concat(dfs)
results.columns = ['seed' , 'time', 'training', 'test', 'validation', 'model', 'constants', 'problem', 'amount', 'variation', 'seed_file']
results["training"] = pd.to_numeric(results["training"])
results["test"] = pd.to_numeric(results["test"])
results["validation"] = pd.to_numeric(results["validation"])
results = results.reset_index(drop=True)
results

,seed,time,training,test,validation,model,constants,problem,amount,variation,seed_file


In [2]:
print(results.problem.unique())
print(results.variation.unique())

[]
[]


In [3]:
# results = results[results.variation.isin(['GGP', 'SC1-GGP', 'SC2-GGP'])]
# results = results[results.variation.isin(['GGP', 'SM1-GGP', 'SM2-GGP'])]
# results

In [4]:
grouped = results.groupby(['problem', 'amount', 'variation'], as_index=False).agg({'training':['min', 'mean', 'median', 'max', 'std']})
grouped

Empty DataFrame
Columns: [(problem, ), (amount, ), (variation, ), (training, min), (training, mean), (training, median), (training, max), (training, std)]
Index: []

In [5]:
for problem in grouped.problem.unique():
    fig, axs = plt.subplots(1, 4, sharey=True, figsize=(20, 8))
    fig.suptitle(problem)
    a = 0
    for amount in grouped.amount.unique():
        if amount.split("_")[0] == problem:
            for variation in grouped.variation.unique():
                hist_dfs = []

                for i, row in results.loc[(results.problem == problem) & (results.amount == amount) & (results.variation == variation)].iterrows():
                    # print('result   s/' + row['problem'] + '/' + row['amount'] + '/' + row['variation'] + '/' + row['seed_file'])
                    hist_df = pd.read_csv(directory + row['problem'] + '/' + row['amount'] + '/' + row['variation'] + '/' + row['seed_file'], sep=";", header=None)
                    hist_df.columns = [row['seed_file'], 'time']
                    hist_dfs.append(hist_df[[row['seed_file']]])

                hist_dfs = pd.concat(hist_dfs, axis=1)
                hist_dfs[variation + " avg"] = hist_dfs.mean(numeric_only=True, axis=1)
                hist_dfs[variation + " std"] = hist_dfs.std(numeric_only=True, axis=1)

                
                axs[a].plot(hist_dfs.index, hist_dfs[variation + " avg"], lw=2, label=variation)
                # axs[a].fill_between(hist_dfs.index, hist_dfs[variation + " avg"] + hist_dfs[variation + " std"], hist_dfs[variation + " avg"] - hist_dfs[variation + " std"], alpha=0.3)
            
            axs[a].set_title(amount)
            axs[a].legend(loc='upper left')
            axs[a].set_xlabel('Gerações')
            axs[a].set_ylabel('MSE')
            axs[a].grid()

            a = a + 1

    plt.savefig(problem + ".svg", format="svg")
    plt.show()
